In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    # !wandb login
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/temporal')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Extra_and_Unused')
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"

Running on colab
/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project


In [7]:
str(get_ipython())

'<google.colab._shell.Shell object at 0x7f46babce940>'

In [2]:
# from __future__ import print_function
# from __future__ import print_function
import os
import numpy as np

from six.moves import xrange
import datetime
import gc

from scipy import signal

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform

#from ResidualStack import ResidualStack
#from Residual import Residual

from Jaguas_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

from datetime import timedelta
import wandb
from wandb import AlertLevel

# wandb.login()

cuda


In [3]:
# root_path = "ConservacionBiologicaIA/Datos/Jaguas_2018"
model_name = f"{root}/temporal/models/model_AE_batch_size_14_num_hiddens_64__day_9_hour_4_final.pth"
config = torch.load(f'temporal/configs/config_AE_batch_size_14_num_hiddens_64__day_9_hour_4.pth', map_location=torch.device('cpu'))
model = AE(num_hiddens=config["num_hiddens"]).to(device)
dataset_test = torch.load(f'temporal/datasets/dataset_test_ae_jaguas_9_70%.pth')
dataset_train = torch.load(f'temporal/datasets/dataset_train_ae_jaguas_9_70%.pth')
model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))

<All keys matched successfully>

In [4]:
# training_loader = DataLoader(dataset_train, batch_size=1)
# test_loader = DataLoader(dataset_test, batch_size=1)
# iterator = iter(training_loader)
# testing = TestModel(model, iterator, device=torch.device("cuda"))
# originals, reconstructions, encodings, label, loss = testing.reconstruct()
# encodings_size = encodings[0].shape

# training_recorder_list = []
# training_hour_list = []
# training_minute_list = []
# delete_samples = []

# for id, item in enumerate(dataset_train):
#     if id% 50 == 0:
#         print(f"id: {id + 1} of {len(dataset_train)}")
#     model.to("cuda")
#     try:
#         originals, reconstructions, encodings, label, loss = testing.reconstruct()
#     except:
#         print(f"error id: {id}")
#         delete_samples.append(id)
#         continue
#     # training_samples_list.append(encodings)
#     training_recorder_list.append(label["recorder"])
#     training_hour_list.append(label["hour"])
#     training_minute_list.append(label["minute"])

# torch.save(training_recorder_list, "training_recorder_list.pth")
# torch.save(training_hour_list, "training_hour_list.pth")
# torch.save(training_minute_list, "training_minute_list.pth")
# training_labels_list = {"recorder": training_recorder_list, "hour": training_hour_list, "minute": training_minute_list}
# torch.save(training_labels_list, "training_labels_list.pth")
# torch.save(delete_samples, "corrupted_samples_list.pth")

In [10]:
training_loader = DataLoader(dataset_train, batch_size=1)
iterator = iter(training_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))
originals, reconstructions, encodings, label, loss, path = testing.reconstruct()

RuntimeError: ignored

In [ ]:
label["recorder"].reshape(label["recorder"].shape[0]*label["recorder"].shape[1])

tensor([19, 19, 19, 19, 19, 52, 52, 52, 52, 52, 24, 24, 24, 24, 24])

In [ ]:
training_loader = DataLoader(dataset_train, batch_size=1)
test_loader = DataLoader(dataset_test, batch_size=1)
iterator = iter(training_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))
encodings_size = encodings[0].shape

training_recorder_list = []
training_hour_list = []
training_minute_list = []
delete_samples = []
training_path_samples = []
training_samples_list_torch = torch.ones(len(dataset_train), 5184).to("cuda")

for id, item in enumerate(dataset_train):
    if id% 500 == 0:
        print(f"id: {id + 1} of {len(dataset_train)}")
    model.to("cuda")
    try:
        originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
    except:
        print(f"error id: {id}")
        delete_samples.append(id)
        continue

    encodings_size = encodings[0].shape
    encodings = encodings.to("cuda").detach()
    encodings = encodings.reshape(encodings.shape[0],
                                encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
    encoding = encodings.squeeze(dim=0)
    # training_samples_list.append(encodings)
    training_samples_list_torch[id] = encodings
    training_recorder_list.append(label["recorder"].reshape(label["recorder"].shape[0]*label["recorder"].shape[1]))
    training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
    training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))
    
    path = 
    training_path_samples.append()

torch.save(training_samples_list_torch, "Features/training_samples_list_torch_70%.pth")
torch.save(training_recorder_list, "Features/training_recorder_list_70%.pth")
torch.save(training_hour_list, "Features/training_hour_list_70%.pth")
torch.save(training_minute_list, "Features/training_minute_list_70%.pth")
training_labels_list = {"recorder": training_recorder_list, "hour": training_hour_list, "minute": training_minute_list}
torch.save(training_labels_list, "Features/training_labels_list_70%.pth")
torch.save(delete_samples, "Features/corrupted_samples_list_70%.pth")

id: 1 of 14047
id: 501 of 14047
id: 1001 of 14047
id: 1501 of 14047
id: 2001 of 14047
